# Complementarity slackness

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import numpy as np
from teaching_optimization.linear_constraints import (
    AllConstraints,
    draw_polyhedron_canonical_form,
    StandardCanonicalForms,
)
from teaching_optimization.simplex_tableau import SimplexAlgorithmTableau
from teaching_optimization.tableau import SimplexTableau


Consider the optimization problem $$\min_{x \in\mathbb{R}^2} -3x_1 - 4x_2$$ subject to
\begin{align*}
-x_1+x_2 & \leq 2,\\
3x_1-2x_2 & \leq 5,\\
x_1+x_2 & \leq 4,\\
x_1,x_2 & \geq 0.
\end{align*}

In [ ]:

A_canonical = np.array([[-1, 1], [3, -2], [1, 1], [-1, 0], [0, -1]])
b_canonical = np.array([2, 5, 4, 0, 0])
polyhedron = AllConstraints.from_canonical_form(matrix=A_canonical, vector=b_canonical)
print(polyhedron)


Plot the polyhedron

In [ ]:
draw_polyhedron_canonical_form(
    matrix_a=A_canonical,
    vector_b=b_canonical,
)


Transform the constraints in standard form

In [ ]:
standard_canonical = StandardCanonicalForms(constraints=polyhedron)


Solve the problem with the simplex algorithm

In [ ]:
objective = np.array([-3, -4, 0, 0, 0])
the_algorithm = SimplexAlgorithmTableau(
    objective=objective,
    constraint_matrix=standard_canonical.standard_matrix,
    right_hand_side=standard_canonical.standard_vector,
)


Solving the problem

In [ ]:
optimal_tableau: SimplexTableau = the_algorithm.solve()


Optimal solution

In [ ]:
print(optimal_tableau.feasible_basic_solution)
x_1 = optimal_tableau.feasible_basic_solution[0]
print(f'x_1 = {x_1:.3g}')
x_2 = optimal_tableau.feasible_basic_solution[1]
print(f'x_2 = {x_2:.3g}')


Note that, for numerical reasons, the actual value of $x_1$ stored in the computer is not exactly 1:

In [ ]:
print(x_1)


Optimal value

In [ ]:
print(f'{optimal_tableau.value_objective_function:.3g}')


Identify the active constraints at the optimal solution. Warning: for the numerical reasons mentioned above,
never check if two float numbers are equal. Instead, use the numpy function `isclose`.

In [ ]:
first_constraint_active = np.isclose(-x_1 + x_2, 2)
print(f'First constraint: {first_constraint_active}')
second_constraint_active = np.isclose(
    3 * x_1 - 2 * x_2, 5
)
print(f'Second constraint: {second_constraint_active}')
third_constraint_active = np.isclose(x_1 + x_2, 4)
print(f'Third constraint: {third_constraint_active}')


# Write the dual of this problem.

The Lagrangian is

$$
L(x_1, x_2, \mu_1, \mu_2, \mu_3, \mu_4, \mu_5) = -3 x_1 - 4 x_2 - \mu_1 x_1 + \mu_1 x_2 - 2 \mu_1
+ 3 \mu_2 x_1 - 2 \mu_2 x_2 - 5 \mu_2
+ \mu_3 x_1 + \mu_3 x_2 - 4 \mu_3
- \mu_4 x_1
- \mu_5 x_2
$$
where $\mu_1, \mu_2, \mu_3, \mu_4, \mu_5 \geq 0$.

For the Lagrangian to be bounded, we need to impose the following constraints:

The coefficient of $x_1$ must be zero:
$$ -3 -\mu_1  + 3 \mu_2 + \mu_3 - \mu_4 = 0, $$
and the coefficient of $x_2$ must be zero:
$$ -4 + \mu_1 - 2 \mu_2  + \mu_3 - \mu_5 = 0.$$

The Lagrangian therefore simplifies to
$$L(x_1, x_2, \mu_1, \mu_2, \mu_3, \mu_4, \mu_5) = -2 \mu_1 -5 \mu_2  - 4 \mu_3. $$

Eliminating $\mu_4$ and $\mu_5$, we can write the dual problem as

$$ \max_{\mu \in \mathbb{R}^3} -2 \mu_1 -5 \mu_2  - 4 \mu_3$$
subject to
$$-\mu_1  + 3 \mu_2 + \mu_3 \geq 3, $$
$$\mu_1 - 2 \mu_2  + \mu_3 \geq 4,$$
$$\mu_1, \mu_2, \mu_3 \geq 0.$$

# Use the complementarity slackness conditions to solve the dual.

If  a variable is non zero, the corresponding
constraint in the dual problem must be active. And if a constraint is
not active, the corresponding variable must be zero.
Here, as the second constraint of the primal is not active, the corresponding
dual variable must be zero: $\mu^*_2=0$.
Also, as the two primal variables $x^*_1$ and $x_2^*$ are non zero, the
corresponding dual constraints must be active, and we have the following
equations:
$$-\mu^*_1  + 3 \mu^*_2 + \mu^*_3 = 3, $$
$$\mu^*_1 - 2 \mu^*_2  + \mu^*_3 = 4.$$

As $\mu^*_2=0$, they simplify to
$$-\mu^*_1 + \mu^*_3 = 3, $$
$$\mu^*_1 + \mu^*_3 = 4.$$

The solution is $\mu^*_1=1/2$ and $\mu^*_3=7/2$.

The value of the objective function is $-15$, as expected from the strong duality theorem.

# Solve the dual with the simplex algorithm to verify the result.

The problem in canonical form is

$$ \min_{\mu \in \mathbb{R}^3} 2 \mu_1 +5 \mu_2  + 4 \mu_3$$
subject to
$$\mu_1  - 3 \mu_2 - \mu_3 \leq -3, $$
$$-\mu_1 + 2 \mu_2  - \mu_3 \leq -4,$$
$$\mu_1, \mu_2, \mu_3 \geq 0$$.

In [ ]:
dual_A_canonical = np.array(
    [[1, -3, -1], [-1, 2, -1], [-1, 0, 0], [0, -1, 0], [0, 0, -1]]
)
dual_b_canonical = np.array([-3, -4, 0, 0, 0])
dual_polyhedron = AllConstraints.from_canonical_form(
    matrix=dual_A_canonical, vector=dual_b_canonical
)
print(polyhedron)



Transform the constraints in standard form

In [ ]:
dual_standard_canonical = StandardCanonicalForms(constraints=dual_polyhedron)


Solve the problem with the simplex algorithm

In [ ]:
dual_objective = np.array([2, 5, 4, 0, 0])
the_dual_algorithm = SimplexAlgorithmTableau(
    objective=dual_objective,
    constraint_matrix=dual_standard_canonical.standard_matrix,
    right_hand_side=dual_standard_canonical.standard_vector,
)


Solving the problem

In [ ]:
dual_optimal_tableau: SimplexTableau = the_dual_algorithm.solve()


Optimal solution

In [ ]:
print(dual_optimal_tableau.feasible_basic_solution)


In [ ]:
mu_1 = dual_optimal_tableau.feasible_basic_solution[0]
print(f'{mu_1=:.3g}')
mu_2 = dual_optimal_tableau.feasible_basic_solution[1]
print(f'{mu_2=:.3g}')
mu_3 = dual_optimal_tableau.feasible_basic_solution[2]
print(f'{mu_3=:.3g}')


Optimal value

In [ ]:
print(
    f'Optimal value of the minimization problem: {dual_optimal_tableau.value_objective_function:.3g}'
)
print(
    f'Optimal value of the maximization problem: {-dual_optimal_tableau.value_objective_function:.3g}'
)

It confirms the above results.